Prompt:

Schreibe einen Python Code, der folgende Funktionen erfüllt:
- Nutzen der CSV mit den Embeddings
- Funktion bauen um eine Frage als text zu embedden
- der embeddete text soll mit den anderen embeddings informationen nach simularity verglichen werden
- es sollen die top 3 nährstens cusens ausgegeben werden

In [4]:
import openai
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import sys
import os
from dotenv import load_dotenv

path_csv_Data = './embeddings.csv'
df = pd.read_csv(path_csv_Data, on_bad_lines='skip')
df['embedding'] = df['embedding'].apply(eval).apply(np.array)
openai.api_key = os.getenv("OPENAI_API_KEY")
model = "gpt-3.5-turbo"

regel1 = ''' Beantworte die Frage so wahrheitsgemäß wie möglich nur mit den Informationen die im KONTEXT stehen. 
    Strukturiere und Formatiere die Antwort leserlich und visuell mit den Informatione die zur verfügung stehen.
    Und ändere den Satzaufbau um den Fragesteller besser zu informieren.
    LINKS und EMAILs die aus dem KONTEXT benutzt werden dürfen NICHT VERÄNDERT werden. '''

regel2: str = '\n Wenn die Antwort nicht im KONTEXT zu finden ist so beantworte die Frage nicht und sage: "Tut mir ' \
    'leid, leider konnte ich zu Ihrer Frage keine Daten finden."'
prepromt = regel1 + regel2


def get_response(message, df, model):


    df["similarities"] = df['embedding'].apply(lambda x: cosine_similarity(x, get_embedding(message, engine="text-embedding-ada-002")))
    top_5_answers = df.sort_values("similarities", ascending=False).iloc[:5]['Sentence'].tolist()
    

        
    sim_kontext = 'KONTEXT: '    
    print('TOP 3 Antworten')
    for i, answer in enumerate(top_5_answers):
        sim_kontext = sim_kontext + '\n' + answer
        print(f"{i + 1}: {answer}")
        
        
    messages = [
    {"role": "system", "content": "Du bist ein hilfreicher Website Q&A Bot." + prepromt},
    {"role": "user", "content": message + '\n' + sim_kontext}
]

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        tempreture=0.5,
    )

    response_text = response["choices"][0]["message"]["content"]

    return response_text


In [5]:


user_query = "Seid wann gibt es die Codersunlimited?"
#user_query = "Was hat die Codersunlimited mit Ryze zu tun?"
#user_query = "Was bietet die Codersunlimited an?"
#user_query = "Wer ist Bunderskanzler in Deutschland?"
print("Frage: " + user_query)

response = get_response(user_query, df, model)
print(response)


Frage: Seid wann gibt es die Codersunlimited?


Erstelle mir aus folgenden code eine schöne Streamlit anwendung, bei der in der sideview erklärt wird was passiert und in der hauptanwendung man die möglichkeit hat seine Frage zustellen. Es soll professionell aussehen: